# Yolov8로 돌리기 위한 전처리 작업

# move image and txt file(from xml to txt)

xml to txt file
* file format: class_id center_x center_y width height (percentage)
* ref: https://roboflow.com/formats/yolov8-pytorch-txt tab: EXAMPLE


image and txt file directory path format
* ref: https://github.com/Balakishan77/Yolov8-Custom-Object-Detetction


train, validation split
1. dataframe으로 만들고, label 컬럼을 만들어서 해당하는 class_id를 넣어놓고
2. fast_ml의 train_valid_test_split 사용하기
3. 나눌 때 resize, padding 해서 저장하기.

In [1]:
from glob import glob
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [5]:
XML_PATH = '/Users/jeonminjeong/Downloads/해양 침적 쓰레기 이미지/Training/Training_labeling_underwater_photo'
TXT_PATH = '/Users/jeonminjeong/Downloads/underwater_images/'

cnt = 0
df = pd.DataFrame()


label2idx = {
    'tire' : 0,
    'spring fish trap' : 1, 'circular fish trap' : 1, 'rectangular fish trap' : 1, 'eel fish trap' : 1,
    'fish net' : 2,
    'wood' : 3,
    'bundle of ropes' : 4, 'bundle of rope' : 4, 'rope' : 4, 
    'other objects' : 5, 'other objets': 5, 'othe objects': 5
    }
 
for xml_file in tqdm(glob(XML_PATH + '/*.xml')): 
    
    filename = f'{TXT_PATH}' + xml_file.split('/')[-1][:-4] + '.txt'
    f = open(filename, 'w')

    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    fname = root.find('filename').text
    for member in root.findall('object'):
        try:
            bbx = member.find('bndbox')
            label = member.find('name').text

            img_width = float(root.find('size')[0].text)
            img_height = float(root.find('size')[1].text)

            xmin = float(bbx.find('xmin').text)
            ymin = float(bbx.find('ymin').text)
            xmax = float(bbx.find('xmax').text)
            ymax = float(bbx.find('ymax').text)
            width = float(bbx.find('width').text) # 
            height = float(bbx.find('height').text) #
            
            label_idx = label2idx[label] 
            x, y = (xmin + xmax)/2/img_width, (ymin + ymax)/2/img_height # center-x, center-y
            w = width / img_width # width
            h = height / img_height # height

            info = f'{label_idx} {x} {y} {w} {h}'
            
            if label_idx != 5:
                f.write(info+'\n')
        except:
            print(xml_file)
            
    if label_idx != 5:
        cnt += 1
        new_row = {'cls_id': label_idx, 'name': fname}
        df = df.append(new_row, ignore_index=True)
    
print('total: ', cnt)

  0%|          | 0/9023 [00:00<?, ?it/s]/var/folders/p4/l6qy0qkd2jd9ptr7xxhfl6cc0000gn/T/ipykernel_4143/3418721082.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/var/folders/p4/l6qy0qkd2jd9ptr7xxhfl6cc0000gn/T/ipykernel_4143/3418721082.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/var/folders/p4/l6qy0qkd2jd9ptr7xxhfl6cc0000gn/T/ipykernel_4143/3418721082.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/var/folders/p4/l6qy0qkd2jd9ptr7xxhfl6cc0000gn/T/ipykernel_4143/3418721082.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

total:  9009


In [6]:
df

,cls_id,name
0,3,wood_048_01220.jpg
1,4,circular fish trap_rope_047_03288.jpg
2,3,wood_048_02729.jpg
3,4,tire_bundle of ropes_042_05398.jpg
4,1,eel fish trap_047_02574.jpg
...,...,...
9004,3,wood_042_01504.jpg
9005,0,tire_044_01433.jpg
9006,0,bundle of ropes_tire_004_01712.jpg
9007,0,tire_029_00862.jpg


In [26]:
!pip install fast_ml 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 1.2 MB/s eta 0:00:00


In [7]:
from fast_ml.model_development import train_valid_test_split

X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target = 'cls_id', method='random', sort_by_col='cls_id',
                                                                            train_size=0.8, valid_size=0.1, test_size=0.1)


print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape)

(7207, 1) (7207,) (901, 1) (901,) (901, 1) (901,)


In [8]:
y_train.groupby(y_train).count()

cls_id
0    1602
1    2032
2     522
3    1382
4    1669
Name: cls_id, dtype: int64

In [9]:
df.groupby('cls_id').count()

,name
cls_id,
0,2003
1,2551
2,650
3,1715
4,2090


In [10]:
X_train

,name
7894,wood_042_07407.jpg
217,tire_044_01238.jpg
4528,spring fish trap_046_03114.jpg
6534,spring fish trap_046_03919.jpg
6153,spring fish trap_046_02999.jpg
...,...
5799,tire_046_02656.jpg
7030,tire_042_08830.jpg
6001,tire_046_03993.jpg
8877,bundle of ropes_circular fish trap_044_04330.jpg


In [89]:
for i in X_train["name"]:
    print(i)
    break

rectangular fish trap_bundle of ropes_047_01634.jpg


In [11]:
import os
import shutil

TRAIN_PATH = '/Users/jeonminjeong/Downloads/underwater_images/train/labels/'
VALID_PATH = '/Users/jeonminjeong/Downloads/underwater_images/valid/labels/'

# move txt file
# train
for i in tqdm(X_train["name"]):
    i = i.split('.')[0] + ".txt"
    shutil.move(TXT_PATH + i, TRAIN_PATH + i)
# valid, test
for i in tqdm(X_valid["name"]):
    i = i.split('.')[0] + ".txt"
    shutil.move(TXT_PATH + i, VALID_PATH + i)
for i in tqdm(X_test["name"]):
    i = i.split('.')[0] + ".txt"
    shutil.move(TXT_PATH + i, VALID_PATH + i)

100%|██████████| 901/901 [00:00<00:00, 14314.33it/s]


14 개의 경우 other objects와 같이 있는 사진이라 삭제함.

0, 1, 2, 3, 4 라벨만 사용. other objects 인 5는 제거.

9023 - 14 = 9009

In [ ]:
# move img file
# 1920 x 1080 -> 640 x 640
import cv2


def SetImageResizeNPadding(image_path, save_image_path, targetWidth, targetHeight, finalTargetSize):
    # 이미지 받을 경로, 저장할 경로, 중간 너비, 중간 높이, 최종 크기(정사각형이므로 길이하나만)
    # load image
    imageToPredict = cv2.imread(image_path, 3) # 단일 이미지 읽기
    img = np.array(imageToPredict); # 이미지 행렬화

    # resize
    img = cv2.resize(imageToPredict, (targetWidth, targetHeight)); # 이미지 리사이즈
    img = np.array(img); # resize 된 이미지 행렬화

    # padding
    y_zero, x_zero, target_height, target_width = (0, 0, img.shape[0], img.shape[1]) # 바뀐 이미지의 패딩계산을 위한 크기

    # 그림 주변에 검은색으로 칠하기
    padding_width = (finalTargetSize - (target_width - x_zero))/2  # w_x = (targetWidth - 그림)을 뺀 나머지 영역 크기 [ 그림나머지/2 [그림] 그림나머지/2 ]
    padding_height = (finalTargetSize - (target_height - y_zero))/2

    if(padding_width < 0):         # 크기가 -면 0으로 지정.
        padding_width = 0
    elif(padding_height < 0):
        padding_height = 0

    M = np.float32([[1, 0, padding_width], [0, 1, padding_height]])  #(2*3 이차원 행렬)
    img_re = cv2.warpAffine(img, M, (finalTargetSize, finalTargetSize)) # 패딩 실행

    # save image
    cv2.imwrite(save_image_path, img_re) # 저장
    
    
# train
TRAIN_IMAGE_PATH = '/Users/jeonminjeong/Downloads/underwater_images/train/images/'
TRAIN_LABEL_PATH = '/Users/jeonminjeong/Downloads/underwater_images/train/labels/'
ORIGINAL_IMAGE_PATH = '/Users/jeonminjeong/Downloads/해양 침적 쓰레기 이미지/Training/Training_underwater_photo/'

# 1920 x 1080 = 640 x 360
for file in tqdm(glob(TRAIN_LABEL_PATH + '*.txt')):
    source = ORIGINAL_IMAGE_PATH + file.split('/')[-1].split('.')[0] + '.jpg'
    destination = TRAIN_IMAGE_PATH + file.split('/')[-1].split('.')[0] + '.jpg'
    # print(source)
    SetImageResizeNPadding(source, destination, 640, 360, 640)
# 7207

In [ ]:
# valid, text
VALID_IMAGE_PATH = '/Users/jeonminjeong/Downloads/underwater_images/valid/images/'
VALID_LABEL_PATH = '/Users/jeonminjeong/Downloads/underwater_images/valid/labels/'

# 1920 x 1080 = 640 x 360
for file in tqdm(glob(VALID_LABEL_PATH + '*.txt')):
    source = ORIGINAL_IMAGE_PATH + file.split('/')[-1].split('.')[0] + '.jpg'
    destination = VALID_IMAGE_PATH + file.split('/')[-1].split('.')[0] + '.jpg'
    # print(source)
    SetImageResizeNPadding(source, destination, 640, 360, 640)
    
# 라벨링이 일치하지 않는, 제대로 되지 않은 eel fish trap_004_02882 삭제: 타이어, fish trap이 있는데 tire가 라벨에 없음.
# 1801
# total 9008

folder template

ref: https://medium.com/augmented-startups/train-yolov8-on-custom-data-6d28cd348262

├── yolov8

 -----└── train

 -------└── images (folder including all training images)

 -------└── labels (folder including all training labels)

 -----└── test

 -------└── images (folder including all testing images)

 -------└── labels (folder including all testing labels)

 -----└── valid

 -------└── images (folder including all testing images)

 -------└── labels (folder including all testing labels)



 underwater.yaml 까지 만들어줍니다.